In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm
import time

# --- 1. Configuration ---
config = {
    "DEVICE": "cuda" if torch.cuda.is_available() else "cpu",
    "BATCH_SIZE": 64,
    "NUM_EPOCHS": 10,
    "LEARNING_RATE": 0.001,
    "DATA_DIR": "/home/kami/Documents/datasets/",
    "NUM_CLASSES": 101
}

print(f"--- SCRIPT 2: STANDARD FINE-TUNING ---")
print(f"Using device: {config['DEVICE']}")
print("-" * 40)

# --- 2. Data Loading ---
data_transforms = transforms.Compose([
    transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
print("Loading Food101 dataset for standard training...")
train_dataset = datasets.Food101(root=config["DATA_DIR"], split='train', download=True, transform=data_transforms)
# Use shuffle=True for standard training
train_loader = DataLoader(
    train_dataset, batch_size=config["BATCH_SIZE"], shuffle=True, num_workers=4, pin_memory=True
)

# --- 3. Model Initialization ---
model = models.vgg16_bn(weights=models.VGG16_BN_Weights.DEFAULT)
# Freeze feature layers
for param in model.features.parameters():
    param.requires_grad = False
# Replace classifier
model.classifier = nn.Sequential(
    nn.Linear(25088, 4096), nn.ReLU(True), nn.Dropout(0.5),
    nn.Linear(4096, 1024), nn.ReLU(True), nn.Dropout(0.5),
    nn.Linear(1024, config["NUM_CLASSES"])
).to(config["DEVICE"])
model = model.to(config["DEVICE"])

# --- 4. Standard Training Loop ---
optimizer = optim.Adam(model.classifier.parameters(), lr=config["LEARNING_RATE"])
criterion = nn.CrossEntropyLoss()
total_training_time = 0

print("--- Starting Standard Fine-Tuning Training ---")
for epoch in range(config["NUM_EPOCHS"]):
    epoch_start_time = time.time()
    model.train() # Set model to training mode
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config['NUM_EPOCHS']}")
    for images, labels in pbar:
        images, labels = images.to(config["DEVICE"]), labels.to(config["DEVICE"])
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        pbar.set_postfix({"loss": f"{loss.item():.4f}"})

    epoch_time = time.time() - epoch_start_time
    total_training_time += epoch_time
    print(f"Epoch {epoch+1} completed in {epoch_time:.2f}s")

# --- 5. Final Report ---
print("-" * 40)
print(f"Finished Standard Training.")
print(f"Total Time for {config['NUM_EPOCHS']} epochs: {total_training_time:.2f} seconds")
print("-" * 40)

--- SCRIPT 2: STANDARD FINE-TUNING ---
Using device: cuda
----------------------------------------
Loading Food101 dataset for standard training...
--- Starting Standard Fine-Tuning Training ---


Epoch 1/10: 100%|██████████| 1184/1184 [02:23<00:00,  8.27it/s, loss=2.5758]


Epoch 1 completed in 143.21s


Epoch 2/10: 100%|██████████| 1184/1184 [02:22<00:00,  8.31it/s, loss=3.1078]


Epoch 2 completed in 142.48s


...

Epoch 8 completed in 144.61s


Epoch 9/10: 100%|██████████| 1184/1184 [02:24<00:00,  8.19it/s, loss=1.7564]


Epoch 9 completed in 144.64s


Epoch 10/10: 100%|██████████| 1184/1184 [02:24<00:00,  8.18it/s, loss=1.8112]

Epoch 10 completed in 144.69s
----------------------------------------
Finished Standard Training.
Total Time for 10 epochs: 1445.04 seconds
----------------------------------------
